## Segmenting and Clustering Assignment

In [2]:
#Import libraries

from bs4 import BeautifulSoup
import requests
import pandas as pd

### I'm scraping the wiki page for the table and pulling out the rows and columns

In [3]:
beautiful = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

soup = BeautifulSoup(beautiful.content, 'lxml')
table = soup.find('table', class_="wikitable sortable")
table_rows = table.find_all('tr')
table_cols = table.find_all('th')

print(table_cols)

### Putting the rows into a dataframe

In [5]:
l = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
df = pd.DataFrame(l, columns=["Postcode", "Borough", "Neighbourhood"])

### Cleaning the dataframe

In [6]:
df=df.drop(index=0)
# Removing unassigned boroughs
df = df.drop(df[(df.Borough == "Not assigned")].index)
# Removing extra \n characters from the web scraping
df.Neighbourhood = df.Neighbourhood.str.replace(pat = "\n", repl = "", regex=False)
df.reset_index(drop=True, inplace=True)
df.head(10)

,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Downtown Toronto,Queen's Park
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern
14,M3B,North York,Don Mills North


In [20]:
# If the Neighbourhood is empty but the Borough isn't then assign the Borough name to the Neighbourhood

for i in df.index:
    if df.Neighbourhood[i] == "Not assigned":
        df.Neighbourhood[i] = df.Borough[i]


### Condensing the dataframe by grouping all Neighbourhoods into Boroughs and Postcodes

In [41]:
df_sorted = df.sort_values('Postcode')
df_sorted.reset_index(drop=True, inplace=True)

for i in df_sorted.index:
    if df_sorted.Postcode[i] == df_sorted.Postcode[i+1] and df_sorted.Borough[i] == df_sorted.Borough[i+1]:
        df_sorted.Neighbourhood[i+1] = df_sorted.Neighbourhood[i] + ", " + df_sorted.Neighbourhood[i+1]
        df_sorted.drop([i], axis=0, inplace=True)
        
# It throws an error but does the job

KeyError: 210

In [45]:
df_sorted.shape

(103, 3)